In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,avg,col,monotonically_increasing_id

spark = SparkSession.builder.appName("GoldlayerCreation").getOrCreate()

# Read Necessary Silver tables
silver_users=spark.read.table("silver.users")
silver_sellers=spark.read.table("silver.sellers")
silver_buyers=spark.read.table("silver.buyers")
silver_countries=spark.read.table("silver.countries")

In [0]:
# Perform the join operations
comprehensive_user_table = silver_users\
    .join(silver_countries,["country"],"outer")\
    .join(silver_buyers,["country"],"outer")\
    .join(silver_sellers,["country"],"outer")

# Select and alias columns from each dataframe to ensure uniqueness
comprehensive_user_table = comprehensive_user_table.select(
    silver_users["country"].alias("Country"),

    # For silver_users
    silver_users["productsSold"].alias("Users_ProductsSold"),
    silver_users["productsWished"].alias("Users_productsWished"),
    silver_users["account_age_years"].alias("Users_account_age_years"),
    silver_users["account_age_group"].alias("Users_account_age_group"),
    silver_users["hasanyapp"].alias("Users_hasanyapp"),
    silver_users["socialnbfollowers"].alias("Users_socialnbfollowers"),
    silver_users["flag_long_title"].alias("Users_flag_long_title"),

    # For silver_countries
    silver_countries["sellers"].alias("Countries_Sellers"),
    silver_countries["topsellers"].alias("Countries_TopSellers"),
    silver_countries["femalesellers"].alias("Countries_FemaleSellers"),
    silver_countries["malesellers"].alias("Countries_MaleSellers"),
    silver_countries["topfemalesellers"].alias("Countries_TopFemaleSellers"),
    silver_countries["topmalesellers"].alias("Countries_TopMaleSellers"),

    # For silver_buyers
    silver_buyers["buyers"].alias("Buyers_Total"),
    silver_buyers["topbuyers"].alias("Buyers_Top"),
    silver_buyers["femalebuyers"].alias("Buyers_Female"),
    silver_buyers["malebuyers"].alias("Buyers_Male"),
    silver_buyers["topfemalebuyers"].alias("Buyers_TopFemale"),
    silver_buyers["topmalebuyers"].alias("Buyers_TopMale"),


    #For silver_sellers
    silver_sellers["nbsellers"].alias("Sellers_Total"),
    silver_sellers["sex"].alias("Sellers_Sex"),
    silver_sellers["meanproductssold"].alias("Sellers_MeanProductsSold"),
    silver_sellers["meanproductslisted"].alias("Sellers_MeanProductsListed"),
    silver_sellers["meanfollowers"].alias("Sellers_MeanFollowers")
)

In [0]:
# comprehensive_user_table.show(5)

+-------+------------------+--------------------+-----------------------+-----------------------+---------------+-----------------------+---------------------+-----------------+--------------------+-----------------------+---------------------+--------------------------+------------------------+------------+----------+-------------+-----------+----------------+--------------+-------------+-----------+------------------------+--------------------------+---------------------+
|Country|Users_ProductsSold|Users_productsWished|Users_account_age_years|Users_account_age_group|Users_hasanyapp|Users_socialnbfollowers|Users_flag_long_title|Countries_Sellers|Countries_TopSellers|Countries_FemaleSellers|Countries_MaleSellers|Countries_TopFemaleSellers|Countries_TopMaleSellers|Buyers_Total|Buyers_Top|Buyers_Female|Buyers_Male|Buyers_TopFemale|Buyers_TopMale|Sellers_Total|Sellers_Sex|Sellers_MeanProductsSold|Sellers_MeanProductsListed|Sellers_MeanFollowers|
+-------+------------------+--------------

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ecom_db_shubham.gold;

In [0]:
comprehensive_user_table.write.repartition(200).mode("overwrite").format("delta").saveAsTable("gold.ecom_one_big_table")